In [1]:
import numpy as np
import pandas as pd
import random
from decision_tree_functions import decision_tree_algorithm, decision_tree_predictions
from sklearn.model_selection import train_test_split
from ipynb.fs.full.features import features_independientes_precio, features_dependientes_precio
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import make_classification


NameError: name 'nltk' is not defined

In [ ]:
df = pd.read_csv("./data/train.csv")

column_names = []
for column in df.columns:
    name = column.replace(" ", "_")
    column_names.append(name)
df.columns = column_names


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [ ]:
df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)

df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)

In [ ]:
y_test = df_test_f['precio']
y_train = df_train_f['precio']
x_test = df_test_f.drop('precio', axis=1)
x_train = df_train_f.drop('precio', axis=1)

In [ ]:
categorias = ['id', 'zona', 'intervalo_metros_cubiertos', 'intervalo_metros_totales', 'provincia', 'escualas_centros_cercanos', 'tipodepropiedad']

categoricos = df_train_f[categorias]
categoricos_test =df_test_f[categorias]
df_full = pd.concat([categoricos_test, categoricos], ignore_index=True)

categoricos = pd.get_dummies(categoricos)
df_ohe_test = pd.get_dummies(df_full).iloc[0:60000]
categoricos_test = df_ohe_test.fillna(0)

In [ ]:
zona = x_train.drop(['titulo', 'descripcion','direccion', 'zona', 'intervalo_metros_cubiertos', 'intervalo_metros_totales', 'provincia', 'escualas_centros_cercanos', 'tipodepropiedad', 'ciudad', 'habitaciones', 'garages', 'banos', 'antiguedad', 'idzona', 'lat', 'lng', 'promedio_precio_hbg_tipo_propiedad_provincia_gen'], axis=1)
df_encoding = pd.merge(zona, categoricos, on= 'id', how= 'left')

zona_test = x_test.drop(['titulo', 'descripcion','direccion', 'zona', 'intervalo_metros_cubiertos', 'intervalo_metros_totales', 'provincia', 'escualas_centros_cercanos', 'tipodepropiedad', 'ciudad', 'habitaciones', 'garages', 'banos', 'antiguedad', 'idzona', 'lat', 'lng', 'promedio_precio_hbg_tipo_propiedad_provincia_gen'], axis=1)
df_encoding_test = pd.merge(zona_test, categoricos_test, on= 'id', how= 'left')
df_encoding_test = df_encoding_test.drop(['tipodepropiedad_Garage', 'tipodepropiedad_Hospedaje'],axis =1)


In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
clf.fit(df_encoding,y_train)

In [ ]:
print(categoricos.shape)
print(categoricos_test.shape)
print(df_encoding.shape)
print(df_encoding_test.shape)

set(df_encoding_test.columns)  - set(df_encoding.columns)

In [ ]:
clf.predict(df_encoding_test)

In [ ]:
clf.feature_importances_

In [ ]:

feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(df_encoding.columns, clf.feature_importances_):
    feats[feature] = importance #add the name/value pair 

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances.sort_values(by='Gini-importance').nlargest(40, 'Gini-importance').plot(kind='bar', rot=90, figsize= (30,30), fontsize = 20)

In [ ]:
cubiertos = importances.transpose()
cubiertos = cubiertos[['metroscubiertos', 'intervalo_metros_cubiertos_Grupo1', 'intervalo_metros_cubiertos_Grupo2', 'intervalo_metros_cubiertos_Grupo3', 'intervalo_metros_cubiertos_Grupo4', 'intervalo_metros_cubiertos_Grupo5']]
cubiertos.transpose()

In [ ]:
totales = importances.transpose()
totales = totales[['metrostotales', 'intervalo_metros_totales_Grupo1', 'intervalo_metros_totales_Grupo2', 'intervalo_metros_totales_Grupo3', 'intervalo_metros_totales_Grupo4', 'intervalo_metros_totales_Grupo5']]
totales.transpose()